In [1]:
from pipeline import CustomModelPipelineCorrected
from custom_models.mlp import MLP  
from helper_methods import get_data_loaders  
from torch import optim
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import torch
import pandas as pd 

# set seaborn style
sns.set_theme(
    context="notebook", style="whitegrid", palette="bright", color_codes=True, rc=None
)

In [ ]:
# --- Configuration Section ---

# Parameters for the search
N_EPOCHS_SEARCH = (
    5  # Max epochs for hyperparameter search runs (early stopping applies)
)
N_EPOCHS_FINAL = 200  # Max epochs for the final model training
ES_PATIENCE = 5  # Early stopping patience

# Define Hyperparameter Search Space
# Experiment 1: Hidden Layer Configurations (Size and Depth combined for simplicity here)
# You can structure this differently if you want strict separation like before
layer_configs = {
    "HL_256_64": (256, 64),
    "HL_512_128": (512, 128),
    "HL_1024_256": (1024, 256),
    "HL_2048_512": (2048, 512),
    "HL_4096_1024": (4096, 1024),
    "HL_1024": (1024,),  # From Depth Exp
    "HL_1024_512_256": (1024, 512, 256),  # From Depth Exp
    # Add more configurations if needed
}

# Experiment 2: Optimizers and Learning Rates
optimizers_config = {
    "Adam_0.005": (optim.Adam, 0.005),
    "Adam_0.001": (optim.Adam, 0.001),
    "RMSprop_0.005": (optim.RMSprop, 0.005),
    "RMSprop_0.001": (optim.RMSprop, 0.001),
}

# Experiment 3: Loss Functions (Optional - if you want to search this)
# criterion_config = {
#     "CrossEntropy": nn.CrossEntropyLoss,
#     "MultiMargin": nn.MultiMarginLoss,
# }
# For now, let's stick to CrossEntropyLoss as indicated by your previous findings
criterion_to_use = nn.CrossEntropyLoss()

In [3]:
# --- Data Loading ---
(
    train_loader,
    train_loader_at_eval,
    test_loader,
    validation_loader,
    n_channels,
    n_classes,
    task,
    pil_dataset,
) = get_data_loaders()

# Calculate input size
sample = next(iter(train_loader))
input_size = n_channels * sample[0].shape[2] * sample[0].shape[3]
num_classes = n_classes
print(f"Input size: {input_size}, Number of classes: {num_classes}")

MedMNIST v3.0.2 @ https://github.com/MedMNIST/MedMNIST/
Using downloaded and verified file: C:\Users\josem\.medmnist\dermamnist.npz
Using downloaded and verified file: C:\Users\josem\.medmnist\dermamnist.npz
Using downloaded and verified file: C:\Users\josem\.medmnist\dermamnist.npz
Using downloaded and verified file: C:\Users\josem\.medmnist\dermamnist.npz
Input size: 2352, Number of classes: 7


In [ ]:
def create_pipeline_for_config(layer_config, opt_class, lr, criterion):
    """Initialize pipeline with MLP model for a specific configuration"""
    # Initialize MLP with proper parameters
    model = MLP(
        input_size=input_size, hidden_sizes=layer_config, num_classes=num_classes
    )

    # Initialize optimizer for this specific model instance
    optimizer = opt_class(model.parameters(), lr=lr)

    pipeline = CustomModelPipelineCorrected(
        model=model,
        criterion=criterion(),  
        optimizer=optimizer,  
        n_epochs=N_EPOCHS_SEARCH, 
        training_data=train_loader,
        validation_data=validation_loader,
        test_data=test_loader,
        patience=ES_PATIENCE,
        min_delta=0.001,
    )
    return pipeline


